# [DevNet Associate Model-Driven Programmability - NETCONF](https://learningnetwork.cisco.com/s/devnet-associate-exam-topics)
### 3.8 Apply concepts of model driven programmability (YANG, RESTCONF, and NETCONF) in a Cisco environment
### 5.1 Describe the value of model driven programmability for infrastructure automation
### 5.10 Interpret the results of a RESTCONF or NETCONF query
### 5.11 Interpret basic YANG models

---

### Tasks:
1. Review Lab Environment.
2. Explore YANG Models Using Advanced NETCONF Explorer (ANX).
3. Interact with IOS-XE YANG Modules Using Python **ncclient**.

---

### Review Lab Environment
The Programmability Foundations Lab has two CSR1000v devices which run IOS-XE 16.2(2a). Access information for these devices is as follows:
1. **Router 1 (R1)** - r1.lab.local (192.168.2.161)
2. **Router 2 (R2)** - r2.lab.local (192.168.2.162)
3. **Username** - wwt
4. **Password** - WWTwwt1!
 
 
 The [Programmability Foundations Lab Guide](https://labs.wwtatc.com/lab-guides/programmability_foundations_lab/index.html) has complete lab topology information.

#### Connect to R1 & R2
1. Click the **MTPuTTY** icon in the taskbar.
2. Expand the **PuTTY Sessions** folder.
3. Double-click on each **r1.lab.local** and **r2.lab.local** to establish SSH sessions.
4. When prompted, log on to both routers.

#### Disable Session Timeout on R2
SSH sessions to **R1** have no time limit although SSH sessions to **R2** expire after 10 minutes of inactivity. Use the following CLI commands to disable session timeout:

```
configure terminal
!
line vty 0 15
 exec-timeout 0
 end
wr mem
```

#### Review R1 NETCONF Configuration
Use the following CLI commands to review information about the NETCONF configuration on R1:

```
show run | include netconf
```

* The result of this command shows the single IOS-XE command **netconf-yang** which enables NETCONF on the device.
* The command **no netconf-yang**, in global configuration (configure terminal) mode will disable NETCONF, platform-wide.

```
show netconf-yang datastores
show netconf-yang status
```

The results of these commands show that:
* The **running** datastore is available.
* NETCONF is enabled on the default port, 830.
* The **candidate** datastore is disabled.

```
show netconf schema
```

This command displays the XML schema for the device and allows you to view a hierarchical representation of the available RPCs and paramaters.

#### Review R2 NETCONF Configuration
Use the following CLI commands to review information about the NETCONF configuration on R2:

```
show run | include netconf
```

* The result of this command shows a line of configuration that is not in the R1 config, **netconf-yang feature candidate-datastore**.
* As the syntax implies, this command enables the **candidate** datastore.
* Use the following commands to validate the status of the **candidate** datastore:

```
show netconf-yang datastores
show netconf-yang status
```

The results of these commands show that:
* The **running** and **candidate** datastores are available.
* NETCONF is enabled on the default port, 830.
* The **candidate** datastore is enabled.
---

### Explore YANG Models Using Advanced NETCONF Explorer (ANX)
ANX allows you to interactively:
* Explore the YANG models of compatible devices.
 * ANX requires devices to support the NETCONF monitoring standard ([RFC6022](https://tools.ietf.org/html/rfc6022)).
* Create NETCONF payloads and filters.
* Send and receive NETCONF RPCs.

#### Connect to R1 with ANX
* ANX requires an online connecction to a network device.
* The lab setup process should have opened a second Chrome browser tab for ANX.
* If not, use the **ANX** desktop shortcut to open a new tab to ANX or manually browse to [http://localhost:9269](http://localhost:9269).

To log on to ANX:
1. Set the **NETCONF Host** field to **r1.lab.local**.
2. Set the **Username** to **wwt**.
3. Set the **Password** to **WWTwwt1!**.
4. Tick the boxes both to **Cache** YANG models and **Remember credentials**.
5. Click the **Login** button.

#### Wait for ANX to Download YANG Models
* ANX will load all of the YANG models from R1 which will take 5-7 minutes.
* The ANX main view will load as soon as the YANG model download completes.

#### Search for YANG Models
* Type **interfaces** in the **Search Models** field and press your **Enter** or **Return** key.
* Notice the search results display all available YANG models which contain the word **interface**, including:
 * Cisco IOS-XE **Native** Operational model.
 * **IETF** Configuration & Operational models.
 * **OpenConfig** models.
* To locate the Cisco IOS-XE **Native** Configuration model, search instead for **native**.

#### Explore YANG Models
* Click on the **ietf-interfaces** model and notice the ANX interface updates to show you details about this specific YANG element.
 * Since we selected a top-level module, ANX displays data for the first container in the hierarcchy.
* Notice details including:
 * The element **type**.
 * The **XPath**.
 * The **Subtree Filter** - **\*\* *this very useful data for Python interactions* \*\***
* These values will update as you navigate YANG models.

#### Explore Device Data
* ANX will show you configuration and state data for a device within the YANG model hierarchy.
* Click the **Show Data** button and choose the **Running** datastore from the menu.
* Expand the **ietf-interfaces** hierarchy including:
 * The **interface** list.
 * The **ipv4** container.
 * The **address** list.
* Notice the configuration data from R1 in the hierarchy.
* Click on the **address** element and notice the ANX display update to show, among other things, the **ietf-ip** namespace.

#### Explore the NETCONF Console
* ANX allows you to send and receive NETCONF payloads directly.
* Click the **ipv4** container in the hierarchyand then click the **NETCONF Console** button.
* In the NETCONF Console view, click the **\<get\>** button and then click the **Send Request** button.
* Notice how the NETCONF Console:
 * Creates a NETCONF XML payload.
 * Sends the payload to the device in an **\<rpc\>** message.
 * Displays the body of the **\<rpc-reply\>** message.
* The NETCONF Console allows you to manually edit XML payloads and choose between **\<get\>**, **\<edit-config\> merge**, **\<edit-config\> delete**, and **\<commit\>** operations.
 * **\*\* *This is another very useful way to build XML payloads for Python interactions* \*\***
---

### Interact with IOS-XE YANG Modules Using Python ncclient.